In [138]:
using DataFrames, CSV, JuMP, Gurobi, LinearAlgebra, Random, Printf, StatsBase, CategoricalArrays, Plots, StatsPlots, Distributions

## ____________________________________________________________________

## Import Datasets (Outputted Probabilities and Returns on $100 Bets)

In [139]:
# add actual data here
xgb_preds = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\notebooks\\xgb_cv_pred.csv",DataFrame)
rf_preds = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\notebooks\\rf_pred.csv",DataFrame)
lgbm_preds = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\notebooks\\lgbm_cv_pred.csv",DataFrame);
logreg_preds = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\notebooks\\logreg_pred.csv",DataFrame);
new_xgb_preds = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\notebooks\\new_xgb_pred.csv",DataFrame);
returns = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\data\\group_stage_betting_odds_final.csv",DataFrame);

In [140]:
returns_matrix = Matrix(returns[:,3:5]);
# matrix of probabilities for model's predictions
xgb_probs = Matrix(xgb_preds[:,3:5]);
rf_probs = Matrix(rf_preds[:,3:5]);
lgbm_probs = Matrix(lgbm_preds[:,3:5]);
logreg_probs = Matrix(logreg_preds[:,3:5]);
new_xgb_probs = Matrix(new_xgb_preds[:,3:5]);

In [141]:
new_xgb_preds

Row,home_team,away_team,home_win,draw,home_loss
,String15,String15,Float64,Float64,Float64
1,Argentina,Mexico,0.691677,0.0415087,0.266814
2,Argentina,Saudi Arabia,0.968919,0.025733,0.00534816
3,Australia,Denmark,0.238239,0.210197,0.551563
4,Belgium,Canada,0.964543,0.0154218,0.0200349
5,Belgium,Morocco,0.86645,0.0244761,0.109074
6,Brazil,Serbia,0.760576,0.0116859,0.227739
7,Brazil,Switzerland,0.907287,0.00921022,0.0835025
8,Cameroon,Brazil,0.00561122,0.175407,0.818982
9,Cameroon,Serbia,0.184903,0.153135,0.661962


In [142]:
returns

Row,home_team,away_team,home_win_earnings_per_dollar,draw_earnings_per_dollar,away_win_earnings_per_dollar,implied_probability_of_home_win,implied_probability_of_draw,implied_probability_of_away_win
,String15,String15,Float64,Float64,Float64,Float64,Float64,Float64
1,Argentina,Mexico,0.549451,2.95,6.12,0.64539,0.338983,0.163399
2,Argentina,Saudi Arabia,0.120048,8.21,24.52,0.892819,0.121803,0.040783
3,Australia,Denmark,6.38,3.88,0.429185,0.6997,0.257732,0.15674
4,Belgium,Canada,0.900901,2.45,3.51,0.526066,0.408163,0.2849
5,Belgium,Morocco,0.581395,3.21,4.92,0.632353,0.311526,0.203252
6,Brazil,Serbia,0.5,3.28,6.6,0.666667,0.304878,0.151515
7,Brazil,Switzerland,0.469484,3.57,6.23,0.680511,0.280112,0.160514
8,Cameroon,Brazil,7.93,4.32,0.359712,0.73545,0.231481,0.126103
9,Cameroon,Serbia,4.3,2.77,0.70922,0.585062,0.361011,0.232558


## ____________________________________________________________________

In [143]:
function FIFA2022_GS_Bettings(P, R, B, K)
    
    # define model
    model = Model(Gurobi.Optimizer)

    # define parameters and decision variables
    m = size(P, 1)              # 48 group stage matches
    n = size(P, 2)              # 3 outcomes

    @variable(model, x[i=1:m, j=1:n] >= 0) # x[i,j] is the amount of money bet on outcome j of match i
    @variable(model, z[i=1:m, j=1:n], Bin) # z[i,j] is whether you bet on outcome j of match i

    # define objective: maximising expected total returns across all matches
    @objective(model, Max, sum(P[i,1]*R[i,1]*x[i,1] + 
                               P[i,2]*R[i,2]*x[i,2] + 
                               P[i,3]*R[i,3]*x[i,3] - 
                               sum(x[i,j] for j=1:n) for i=1:m))

    # # constraints
    # 1) You can only bet on one outcome for each match
    @constraint(model, [i=1:m], sum(z[i,j] for j=1:n) <= 1)

    # 2) x[i,j] <= B*z[i,j] for all i and j
    @constraint(model, [i=1:m, j=1:n], x[i,j] <= B/12*z[i,j])

    # 3) Budget constraints
    @constraint(model, sum(x[i,j] for j=1:n, i=1:m) <= B)

    # 4) You must spread your risk across at least K matches
    @constraint(model, sum(z[i,j] for j=1:n, i=1:m) >= K)

    # 5) if z = 1, then x > 0
    @constraint(model, [i=1:m, j=1:n], x[i,j] >= 1*z[i,j])

    # solve model
    optimize!(model)

    # get optimal bet amounts (x) and how you bet (z)
    opt_val = objective_value(model)
    x_opt = value.(x)
    z_opt = value.(z)

    return opt_val, x_opt, z_opt
end

FIFA2022_GS_Bettings (generic function with 1 method)

In [144]:
exp_prof, x_opt, z_opt = FIFA2022_GS_Bettings(new_xgb_probs, returns_matrix, 5000, 5);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-16
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads
Optimize a model with 338 rows, 288 columns and 1008 nonzeros
Model fingerprint: 0xd4f2838f
Variable types: 144 continuous, 144 integer (144 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [1e-02, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+03]
Found heuristic solution: objective 871.3801802
Presolve removed 250 rows and 180 columns
Presolve time: 0.00s
Presolved: 88 rows, 108 columns, 332 nonzeros
Found heuristic solution: objective 872.4244194
Variable types: 30 continuous, 78 integer (78 binary)
Found heuristic solution: objective 872.4910645

Root relaxation: objective 1.607237e+04, 19 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Wo

In [145]:
# which matches to bet on
betting_matches = findall(x -> x >= 1, z_opt);

In [146]:
# make betting matches into list of first index of tuples
betting_matches_list = [x[1] for x in betting_matches]

12-element Vector{Int64}:
 28
 29
 33
 34
 43
 11
 35
 41
 19
 31
 44
 45

In [147]:
x_opt[31,:]

3-element Vector{Float64}:
   0.0
   0.0
 416.66666666666663

In [148]:
outcomes = new_xgb_preds[betting_matches_list,:]

Row,home_team,away_team,home_win,draw,home_loss
,String15,String15,Float64,Float64,Float64
1,Morocco,Croatia,0.465358,0.434983,0.0996595
2,Netherlands,Ecuador,0.835085,0.113837,0.051078
3,Portugal,Ghana,0.805563,0.178387,0.0160498
4,Portugal,Uruguay,0.903218,0.0645231,0.0322593
5,Tunisia,Australia,0.625646,0.188217,0.186137
6,Costa Rica,Germany,0.0131862,0.444052,0.542761
7,Qatar,Ecuador,0.326202,0.570737,0.103061
8,Spain,Germany,0.227806,0.54764,0.224554
9,France,Denmark,0.0639191,0.0319283,0.904153


In [150]:
outcomes[:,"bet_amount"] = [x_opt[i] for i in betting_matches];

In [151]:
outcomes[:,"actual_outcome"] = [2,2,1,1,3,3,3,2,1,3,1,2];

In [152]:
outcomes[:,"bet_outcome"] = [x[2] for x in betting_matches];

In [153]:
# is actual_outcome = bet_outcome?
outcomes[:,"correct"] = outcomes[:,"actual_outcome"] .== outcomes[:,"bet_outcome"];

In [154]:
# bet result: if correct == false, then bet_result = -bet_amount, if correct == true, then bet_result = bet_amount*
bet_result = exp_prof - sum(outcomes[:,"bet_amount"].* (outcomes[:,"correct"].==false))

12739.040936208332

In [159]:
betting_matches

12-element Vector{CartesianIndex{2}}:
 CartesianIndex(28, 1)
 CartesianIndex(29, 1)
 CartesianIndex(33, 1)
 CartesianIndex(34, 1)
 CartesianIndex(43, 1)
 CartesianIndex(11, 2)
 CartesianIndex(35, 2)
 CartesianIndex(41, 2)
 CartesianIndex(19, 3)
 CartesianIndex(31, 3)
 CartesianIndex(44, 3)
 CartesianIndex(45, 3)

Row,home_team,away_team,home_win_earnings_per_dollar,draw_earnings_per_dollar,away_win_earnings_per_dollar,implied_probability_of_home_win,implied_probability_of_draw,implied_probability_of_away_win
,String15,String15,Float64,Float64,Float64,Float64,Float64,Float64
28,Netherlands,Qatar,13.51,6.0,0.210084,0.826389,0.166667,0.0740192


In [155]:
outcomes["return_outcome_for_bet"] = returns

Row,home_team,away_team,home_win,draw,home_loss,bet_amount,actual_outcome,bet_outcome,correct
,String15,String15,Float64,Float64,Float64,Float64,Int64,Int64,Bool
1,Morocco,Croatia,0.465358,0.434983,0.0996595,416.667,2,1,false
2,Netherlands,Ecuador,0.835085,0.113837,0.051078,416.667,2,1,false
3,Portugal,Ghana,0.805563,0.178387,0.0160498,416.667,1,1,true
4,Portugal,Uruguay,0.903218,0.0645231,0.0322593,416.667,1,1,true
5,Tunisia,Australia,0.625646,0.188217,0.186137,416.667,3,1,false
6,Costa Rica,Germany,0.0131862,0.444052,0.542761,416.667,3,2,false
7,Qatar,Ecuador,0.326202,0.570737,0.103061,416.667,3,2,false
8,Spain,Germany,0.227806,0.54764,0.224554,416.667,2,2,true
9,France,Denmark,0.0639191,0.0319283,0.904153,416.667,1,3,false


## money actually earned